# 机器学习基础  -- 使用决策树对优惠券情况做出预测

为了贴近实际生活和应用，以实际数据集的处理为主。在2016年1月1日至2016年6月30日之间真实线上线下消费行为，预测用户在2016年7月领取优惠券后15天以内的使用情况。 
注意： 为了保护用户和商家的隐私，所有数据均作匿名处理，同时采用了有偏采样和必要过滤。

## 数据集 ccf_offline_stage1_train.csv（训练数据）
Field | Description
:-|-
User_id | 用户ID
Merchant_id | 商户ID
Coupon_id | 优惠券ID：null表示无优惠券消费，此时Discount_rate和Date_received字段无意义
Discount_rate | 优惠率：x \in [0,1]代表折扣率；x:y表示满x减y。单位是元
Distance | user经常活动的地点离该merchant的最近门店距离是x*500米（如果是连锁店，则取最近的一家门店），x$\in[0,10]$；null表示无此信息，0表示低于500米，10表示大于5公里；
Date_received | 领取优惠券日期
Date | 消费日期：如果Date=null & Coupon_id != null，该记录表示领取优惠券但没有使用，即负样本；如果Date!=null & Coupon_id = null，则表示普通消费日期；如果Date!=null & Coupon_id != null，则表示用优惠券消费日期，即正样本；

In [1]:
# load plugin
import pandas as pd
import numpy as np

In [2]:
#  load data
train_data = pd.read_csv('../../resource/ccf_offline_stage1_train.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   User_id        int64  
 1   Merchant_id    int64  
 2   Coupon_id      float64
 3   Discount_rate  object 
 4   Distance       float64
 5   Date_received  float64
 6   Date           float64
dtypes: float64(4), int64(2), object(1)
memory usage: 93.7+ MB


In [3]:
train_data.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [4]:
print(train_data.shape)

data = train_data.dropna(how = 'any')
print(train_data.shape)

(1754884, 7)
(1754884, 7)


Discount_rate是object类型的，object在pandas中代表字符串，字符串类型不能输入模型中，所以需要改为数值类型

In [5]:
print('Discount_rate 类型: \n', data['Discount_rate'].unique())
# [0,1] 表示折扣率
# x:y 表示满 x 减 y

Discount_rate 类型: 
 ['20:1' '20:5' '30:5' '50:10' '10:5' '50:20' '100:10' '30:10' '50:5'
 '30:1' '100:30' '0.8' '200:30' '100:20' '10:1' '200:20' '0.95' '5:1'
 '100:5' '100:50' '50:1' '20:10' '150:10' '0.9' '200:50' '150:20' '150:50'
 '200:5' '300:30' '100:1' '200:10' '150:30' '0.85' '0.6' '0.5' '300:20'
 '200:100' '300:50' '150:5' '300:10' '0.75' '0.7' '30:20' '50:30']


## 将 Discount_rate 转化为数值特征

### 打折类型
### x:y 表示满 x 减 y          将 x:y 类型的字符串设为1
### [0,1] 表示折扣率           将 [0,1] 类型的字符串设为 0 

In [6]:
def getDiscountType(row):
    if ':' in row:
        return 1
    else:
        return 0

data['Discount_rate'] = data['Discount_rate'].apply(getDiscountType)

/Users/lilithgames/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67165 entries, 6 to 1754880
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   User_id        67165 non-null  int64  
 1   Merchant_id    67165 non-null  int64  
 2   Coupon_id      67165 non-null  float64
 3   Discount_rate  67165 non-null  int64  
 4   Distance       67165 non-null  float64
 5   Date_received  67165 non-null  float64
 6   Date           67165 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 4.1 MB


In [8]:
# load plugin

In [9]:
# 导入DecisionTreeClassifier模型
from sklearn.tree import DecisionTreeClassifier
# 导入 train_test_split，用于划分数据集和测试集
from sklearn.model_selection import train_test_split
# 导入 accuracy_score 准确指标
from sklearn.metrics import accuracy_score

# add label row to the dataset

### 标注标签Label  标注哪些样本是正样本 y=1，哪些是负样本 y = -1
### 预测目标：用户在领取优惠券之后 15 之内的消费情况
### (Date - Date_received <= 15) 表示领取优惠券且在15天内使用，即正样本，y = 1
### (Date - Date_received > 15)   表示领取优惠券未在15天内使用，即负样本，y = 0
### pandas 关于时间的教程 ```https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html```

In [10]:
def label(row):
    if row['Date'] != 'null':
        td = pd.to_datetime(row['Date'], format = '%Y%m%d') - pd.to_datetime(row['Date_received'], format = '%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

data['label'] = data.apply(label, axis = 1)

/Users/lilithgames/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 统计正负样本

In [11]:
print(data['label'].value_counts())

1    57060
0    10105
Name: label, dtype: int64


## 划分数据集 80%训练集 20%测试集

In [12]:
# 80% train 20% test
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size = 0.2, random_state=3)

In [13]:
## 查验训练样本的数量和类别分布
y_train.value_counts()

2751537    96
6641735    86
6929894    80
501441     59
2839484    56
           ..
5679946     1
2208346     1
1273834     1
3395905     1
4461556     1
Name: User_id, Length: 34984, dtype: int64

In [14]:
# 查验测试样本的数量和类别分布
y_test.value_counts()

6641735    27
2751537    22
2839484    15
2507268    14
501441     14
           ..
4284393     1
1242446     1
5408744     1
2577930     1
89464       1
Name: User_id, Length: 11405, dtype: int64

In [15]:
## 初始化分类决策树模型， 深度为5层

In [16]:
model = DecisionTreeClassifier(max_depth=6, random_state = 1)

In [17]:
# 模型训练

In [18]:
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, random_state=1)

In [19]:
# 模型预测

In [20]:
y_pred = model.predict(X_test)

In [21]:
# 模型评估

In [22]:
accuracy_score(y_test, y_pred)

0.011315417256011316

In [23]:
# 将模型选择特征的标准改为entropy

In [24]:
model = DecisionTreeClassifier(criterion='entropy', random_state=1, max_depth=2)

In [25]:
# 模型训练
model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=1)

In [26]:
# 预测
y_pred = model.predict(X_test)

In [27]:
# 评估
accuracy_score(y_test, y_pred)

0.0040943944018462

In [28]:
### 除以上关键步骤，同学也可以自行对数据进行探索，以及其他任何形式对特征预处理方法和特征工程处理。希望同学们能够通过此次作业，着重理解机器学习任务的开展流程，后续我们会讲到更多提升模型效果的方法。对于数据处理的技能和方法，鼓励大家课下多投入时间去探索。祝大家学习进步！

In [29]:
# iris

In [30]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz 

In [31]:
iris = load_iris()
X = iris.data 
y = iris.target
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X, y)

DecisionTreeClassifier()

In [46]:
export_graphviz(
            tree_clf,
            out_file="../../resource/course_data/iris_tree.dot",
            feature_names=iris.feature_names,
            class_names=iris.target_names,
            rounded=True,
            filled=True)

FileNotFoundError: [Errno 2] No such file or directory: '../resource/course_data/iris_tree.dot'

In [33]:
for line in open('../../resource/course_data/iris_tree.dot'):
    print(line)

digraph Tree {

node [shape=box, style="filled, rounded", color="black", fontname=helvetica] ;

edge [fontname=helvetica] ;

0 [label="petal length (cm) <= 2.45\ngini = 0.667\nsamples = 150\nvalue = [50, 50, 50]\nclass = setosa", fillcolor="#ffffff"] ;

1 [label="gini = 0.0\nsamples = 50\nvalue = [50, 0, 0]\nclass = setosa", fillcolor="#e58139"] ;

0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;

2 [label="petal width (cm) <= 1.75\ngini = 0.5\nsamples = 100\nvalue = [0, 50, 50]\nclass = versicolor", fillcolor="#ffffff"] ;

0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;

3 [label="petal length (cm) <= 4.95\ngini = 0.168\nsamples = 54\nvalue = [0, 49, 5]\nclass = versicolor", fillcolor="#4de88e"] ;

2 -> 3 ;

4 [label="petal width (cm) <= 1.65\ngini = 0.041\nsamples = 48\nvalue = [0, 47, 1]\nclass = versicolor", fillcolor="#3de684"] ;

3 -> 4 ;

5 [label="gini = 0.0\nsamples = 47\nvalue = [0, 47, 0]\nclass = versicolor", fillcolor="#39e581"] ;

4 -> 5 ;

6 

In [34]:
## Salient Features

In [35]:
tree_clf.feature_importances_

array([0.01333333, 0.        , 0.56405596, 0.42261071])

In [36]:
## Build Decision Tree: CART

In [37]:
import pandas as pd

In [38]:
mock_data = {
    'gender':['F', 'F', 'F', 'F', 'M', 'M', 'M'],
    'income': ['+10', '-10', '+10', '+10', '+10', '+10', '-10'],
    'family_number': [1, 1, 2, 1, 1, 1, 2],
    'bought': [1, 1, 1, 0, 0, 0, 1],
}

In [39]:
dataset = pd.DataFrame.from_dict(mock_data)

In [40]:
import numpy as np
from collections import Counter

In [41]:
def entropy(elements):
    counter = Counter(elements)
    probabilities = [counter[e] / len(elements) for e in elements]
    return -sum(p * np.log10(p) for p in probabilities)

In [42]:
def find_the_min_spilter(training_data: pd.DataFrame, target: str) -> str:
    x_fields = set(training_data.columns.tolist()) - {target}
    
    spliter = None
    min_entropy = float('inf')
    
    for f in x_fields:
        elements = set(training_data[f])
        for e in elements:
            sub_spliter_1 = training_data[dataset[f] == e][target].tolist()
            entropy_1 = entropy(sub_spliter_1)
            sub_spliter_2 = training_data[dataset[f] != e][target].tolist()
            entropy_2 = entropy(sub_spliter_2)
            entropy_v = entropy_1 + entropy_2
            
            if entropy_v < min_entropy:
                min_entropy = entropy_v
                spliter = (f, e)
    
    print('spliter is: {}'.format(spliter))
    print('the min entropy is: {}'.format(min_entropy))
    
    return spliter

In [43]:
find_the_min_spilter(dataset, 'bought')

spliter is: ('income', '+10')
the min entropy is: 0.7176797562470717


('income', '+10')

In [44]:
dataset[dataset['income'] == '-10']

,gender,income,family_number,bought
1,F,-10,1,1
6,M,-10,2,1


In [45]:
dataset[dataset['income'] != '-10']

,gender,income,family_number,bought
0,F,+10,1,1
2,F,+10,2,1
3,F,+10,1,0
4,M,+10,1,0
5,M,+10,1,0
